## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [ ]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [ ]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

#### `tone_color_converter` 
- parameter
    - config_path : config(구성) 파일 경로. `config.json` 파일에 정의된 형식대로 converter 구성
    - device : inference시 사용할 장치 정의. `cuda:0` 이 기본값. `cuda`면 gpu, `cpu면` cpu 사용
- method
    - load_ckpt
        - ```python
            def load_ckpt(self, ckpt_path):
            checkpoint_dict = torch.load(ckpt_path, map_location=torch.device(self.device))
            a, b = self.model.load_state_dict(checkpoint_dict['model'], strict=False)
            print("Loaded checkpoint '{}'".format(ckpt_path))
            print('missing/unexpected keys:', a, b)
            ```
        - `api.py`에 정의되어 있음
        - `tone_color_converter`의 부모 클래스에 정의된 메서드.
        - 가중치 파일을 torch.load()로 로드하고 앞서 정의한 `device`에 값을 저장한다.

- 이후에 출력 파일 디렉토리를 생성한다.

### Obtain Tone Color Embedding
`checkpoints_v2/ses` 폴더에서 바로 tone color embedding 로드 가능

#### 예시 음성 주의 사항
- 반드시 한 사람의 목소리를 입력해야함
- 소음이 녹음되지 않게 주의해야함
- 충분히 긴 시간이 녹음되어야함 (약 1분)
- 발화 간에 공백이 길면 안됨

### 🔥주의🔥
- 아래 코드를 실행하기 위해서는 cuda, cudnn을 설치해야한다
- `conda install -c conda-forge cudnn=8.1.0`를 실행해서 cuda, cudnn을 둘 다 설치하자.

In [ ]:
reference_speaker = 'resources/openvoice_2.mp3' # 클론할 목소리가 들어갈 자리
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

#### 코드 설명
`get_se`
- parameter
    - `audio_path` : 음성 파일 경로
    - `vc_model` : voice_converter model
    - `target_dir`(Optional) : 음성에서 tone color 가중치 추출 후 저장할 디렉토리. 디폴트로 'processed'로 설정됨.
    - `vad` (Optional) : True 설정 시에는 Vad를 사용하고, False로 설정하면 whisper를 사용한다.


- 내부 동작
    - audio_name
        - 주어진 오디오 파일 경로에서 파일 이름을 추출하고, 버전 및 해시 값을 조합하여 고유한 이름을 생성
        - 즉, 경로에서 이름 추출하고 해시 값 추가해서 고유하게 만듦
    - se_path 
        - `target_dir` + `audio_name` + `se.pth` 로 가중치의 경로를 생성함.
    - vad 혹은 whisper를 사용해서 audio segment가 생성된다. 만약 segment 길이가 0이 되면 예외 처리 실행


- 반환값
    - speaker embedding과 audio_name
    - `vc_model`의 `extract_se` 메서드로 segment embedding을 생성합니다.
        - 우선 `librosa`로 segment를 로드하고 audio 데이터를 torch.FloatTensor로 변환
        - device로 텐서 이동
        - `unsqueeze(0)` 으로 차원 확장
        - 스펙토그램 계산
        - 스펙토그램을 encoder에 넣어서 audio segment 별로 Embedding 생성한다.
        - 이후 embeddeing 값들을 `torch.stack`으로 쌓은 다음 원소별 평균을 구해서 최종 임베딩 값을 구한다.

#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

### 🔥주의🔥
1. 이 [경로](https://visualstudio.microsoft.com/ko/visual-cpp-build-tools/)에서 Microsoft C++ build tool 설치
2. Microsoft Visual C++ 14.0 + Microsoft C++ Build Tools 이상 설치 

In [ ]:
from melo.api import TTS

texts = {
    'EN_NEWEST': """The left block is the encoder, and the right block is the decoder. 
                    If you don’t understand the parts of this model yet, 
                    I highly recommend going over Harvard’s “The Annotated Transformer” guide 
                    where they code the transformer model in PyTorch from scratch. """,  # The newest English base speaker model
    'EN': """The left block is the encoder, and the right block is the decoder. 
                    If you don’t understand the parts of this model yet, 
                    I highly recommend going over Harvard’s “The Annotated Transformer” guide 
                    where they code the transformer model in PyTorch from scratch. """,
    'ES': """El bloque izquierdo es el codificador y el bloque derecho es el decodificador. Si aún no comprende las partes de este modelo, 
            le recomiendo que consulte la guía "The Annotated Transformer" de Harvard, donde codifican el modelo del transformador en PyTorch desde cero.""",
    'FR': """Le bloc de gauche est l'encodeur et le bloc de droite est le décodeur. Si vous ne comprenez pas encore les parties de ce modèle, je vous recommande 
            fortement de consulter le guide « The Annotated Transformer » de Harvard où ils codent le modèle de transformateur dans PyTorch à partir de zéro.""",
    'ZH': """左边的块是编码器，右边的块是解码器。如果您还不了解该模型的各个部分，我强烈建议您阅读哈佛大学的“带注释的 Transformer”指南，其中他们从头开始在 PyTorch 中编码了 Transformer 模型。""",
    'JP': """左側のブロックはエンコーダー、右側のブロックはデコーダーです。このモデルの各部分をまだ理解していない場合は、
            PyTorch でトランスフォーマー モデルを最初からコーディングしているハーバード大学の「注釈付きトランスフォーマー」ガイドを読むことを強くお勧めします""",
    'KR': """배고프지 않지만 무언가를 먹고싶고 피곤하지 않지만 누워서 자고싶다. 운동하고 싶지만 운동을 가기귀찮고 게임을 하고싶다""",
}


src_path = f'{output_dir}/tmp.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)